### 1. Import Required Libraries and Read in Data


#### 1.1 Import necessary libraries 

In [1]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, PandasTools
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from deepchem import metrics

#### 1.2 Read in the BBB dataset from Adenot paper

In [2]:
bbbneg = pd.read_csv("Adenot_BBB-.csv", names=['Drug', 'SMILES'])
bbbneg['permeable'] = np.zeros(len(bbbneg))
bbbneg

,Drug,SMILES,permeable
0,ACEBUTOLOL,CCCC(=O)Nc1ccc(c(c1)C(C)=O)OCC(O)CNC(C)C,0.0
1,DACTINOMYCIN,CC(C)[C@H]1NC(=O)[C@@H](NC(=O)c2ccc(c3c2N=C2C(...,0.0
2,ALDOSTERONE,C[C@@]12CCC(=O)C=C2CC[C@H]2C3CC[C@H](C(=O)CO)C...,0.0
3,AMILORIDE,N\C(=N)\NC(=O)c1nc(c(nc1N)N)Cl,0.0
4,AMIODARONE,CCCCc1oc2ccccc2c1C(=O)c1cc(c(c(c1)[I])OCCN(CC)...,0.0
...,...,...,...
350,KETOCONAZOLE,CC(=O)N1CCN(CC1)c1ccc(cc1)OCC1CO[C@@](Cn2ccnc2...,0.0
351,SPIRAMYCIN,CO[C@H]1[C@H](O)CC(=O)O[C@H](C)CC=CC=C[C@H](OC...,0.0
352,TEICOPLANIN,COC(=O)C1NC(=O)[C@H]2NC(=O)[C@H](NC(=O)[C@@H]3...,0.0
353,TERFENADINE,CC(C)(C)c1ccc(cc1)C(O)CCCN1CCC(CC1)C(O)(c1cccc...,0.0


In [3]:
bbbpos = pd.read_csv("Adenot_BBB+.csv", names=['Drug', 'SMILES'])
bbbpos['permeable'] = np.ones(len(bbbpos))
bbbpos

,Drug,SMILES,permeable
0,ADENOSYL-METHIONINE,C[S+](CC[C@H](N)C([O-])=O)C[C@H]1O[C@H]([C@H](...,1.0
1,ACEGLUTAMIDE,CC(=O)N[C@@H](CCC(N)=O)C(O)=O,1.0
2,ATROMEPINE,CN1C2CCC1CC(C2)OC(=O)C(C)(CO)c1ccccc1,1.0
3,ATROPINE,CN1[C@@H]2CC[C@H]1CC(C2)OC(=O)C(CO)c1ccccc1,1.0
4,BENZATROPINE,CN1[C@H]2CC[C@@H]1CC(C2)OC(c1ccccc1)c1ccccc1,1.0
...,...,...,...
1332,AMITRIPTYLINE,CN(C)CCC=C1c2ccccc2CCc2ccccc12,1.0
1333,CHLORPROMAZINE,CN(C)CCCN1c2ccccc2Sc2ccc(cc12)Cl,1.0
1334,DISULFIRAM,CCN(CC)C(=S)SSC(=S)N(CC)CC,1.0
1335,MORPHINE,CN1CC[C@@]23[C@H]4Oc5c(ccc(c52)C[C@@H]1[C@@H]3...,1.0


### 2. Data Preprocessing

In [4]:
bbb_df = pd.concat([bbbneg, bbbpos])
print(bbb_df.shape)
bbb_df.reset_index(inplace=True, drop=True)

(1692, 3)


In [5]:
#delete missing data
print('Number of NA: ', bbb_df[['Drug', 'SMILES', 'permeable']].isna().sum())
bbb_df = bbb_df.dropna(subset = ['SMILES'])
print(bbb_df.shape)

Number of NA:  Drug         0
SMILES       1
permeable    0
dtype: int64
(1691, 3)


In [6]:
bbb_df[bbb_df['Drug'] == 'COBALAMINE'].values

array([], shape=(0, 3), dtype=object)

In [7]:
clean_smiles = []

for smile in bbb_df['SMILES']:
  clean = max(str(smile).split(" "), key = len) # split the smiles by " "due to strange dataset entries, and keep the longest substring
  clean_smiles.append(clean)
  
bbb_df['SMILES'] = clean_smiles

In [8]:
# updated_smiles = bbb_df["SMILES"]
# updated_count = 0

# for smile in bbb_df['SMILES']:
#   if "." in updated_smiles[i]:
#     updated_count += 1
# print("After cleaning there is {} molecule with salts in their smiles.".format(updated_count))

In [9]:
#multiple components
multiple_components = [smile for smile in bbb_df['SMILES'] if '.' in smile]
print('Number with Multiple Components: ', len(multiple_components))

Number with Multiple Components:  0


In [10]:
#remove duplicates
print('There are {} duplicates in BBB dataset.'.format(sum(bbb_df.duplicated('SMILES'))))
bbb_df.drop_duplicates(subset='SMILES', keep = 'first', inplace = True, ignore_index=True)
bbb_df.reset_index(drop = True, inplace = True)
print('After cleaning, there are {} duplicates left in BBB dataset.'.format(sum(bbb_df.duplicated('SMILES'))))

There are 29 duplicates in BBB dataset.
After cleaning, there are 0 duplicates left in BBB dataset.


In [11]:
bbb_df['permeable'] = bbb_df['permeable'].astype(int)

#### 2.1 Molecule encoding

use the Morgan Fingerprints (radius = 2, nBits = 1024) as our molecular descriptors

In [12]:
r = 2
d = 1024

fp_list = []
mol_list = []
for smile in bbb_df['SMILES']:
    mol = Chem.MolFromSmiles(smile)
    if mol is None:
        print('invalid mol')
        bbb_df = bbb_df[bbb_df.SMILES != smile]
    else: 
        fp = AllChem.GetMorganFingerprintAsBitVect(mol,r,nBits=d,useChirality=False)
        arr = np.zeros((0,), dtype=np.int8)
        DataStructs.ConvertToNumpyArray(fp, arr)
        fp_list.append(arr)
        mol_list.append(mol)

fp_df = pd.DataFrame(fp_list)
morgan_list = np.array(fp_list)
bbb_df = pd.concat([bbb_df, fp_df], axis=1)
#bbb_df['molecule'] = mol_list
print(morgan_list.shape)
print(bbb_df.shape)
bbb_df

[20:03:02] SMILES Parse Error: syntax error while parsing: HYDROXYAMPHETAMINE
[20:03:02] SMILES Parse Error: Failed parsing SMILES 'HYDROXYAMPHETAMINE' for input: 'HYDROXYAMPHETAMINE'
[20:03:02] SMILES Parse Error: syntax error while parsing: CO[C@]1(NC(=O)CSC[C@@H]([N+H3])C([O-])=O)[C@H]2SCC(=C(N2C1=O)C(O)=O)CSc1nnnn1C
[20:03:02] SMILES Parse Error: Failed parsing SMILES 'CO[C@]1(NC(=O)CSC[C@@H]([N+H3])C([O-])=O)[C@H]2SCC(=C(N2C1=O)C(O)=O)CSc1nnnn1C' for input: 'CO[C@]1(NC(=O)CSC[C@@H]([N+H3])C([O-])=O)[C@H]2SCC(=C(N2C1=O)C(O)=O)CSc1nnnn1C'


invalid mol
invalid mol
(1660, 1024)
(1662, 1027)


,Drug,SMILES,permeable,0,1,2,3,4,5,6,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,ACEBUTOLOL,CCCC(=O)Nc1ccc(c(c1)C(C)=O)OCC(O)CNC(C)C,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,DACTINOMYCIN,CC(C)[C@H]1NC(=O)[C@@H](NC(=O)c2ccc(c3c2N=C2C(...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,ALDOSTERONE,C[C@@]12CCC(=O)C=C2CC[C@H]2C3CC[C@H](C(=O)CO)C...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,AMILORIDE,N\C(=N)\NC(=O)c1nc(c(nc1N)N)Cl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AMIODARONE,CCCCc1oc2ccccc2c1C(=O)c1cc(c(c(c1)[I])OCCN(CC)...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,DISULFIRAM,CCN(CC)C(=S)SSC(=S)N(CC)CC,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1660,MORPHINE,CN1CC[C@@]23[C@H]4Oc5c(ccc(c52)C[C@@H]1[C@@H]3...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1661,TRIFLUOPERAZINE,CN1CCN(CCCN2c3ccccc3Sc3ccc(cc23)C(F)(F)F)CC1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#delete missing data after obtaining fingerprints
print('Number of NA: ', bbb_df.isna().sum())
bbb_df = bbb_df.dropna()
print(bbb_df.shape)

Number of NA:  Drug         2
SMILES       2
permeable    2
0            2
1            2
            ..
1019         2
1020         2
1021         2
1022         2
1023         2
Length: 1027, dtype: int64
(1658, 1027)


In [14]:
bbb_df.to_csv('adenot_processed.csv', index=False)

In [15]:
bbb_df.iloc[:, 3:]

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1656,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [16]:
# Please write your code below:
X = bbb_df.iloc[:,3:].copy()
y = bbb_df.iloc[:,2].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### 3. Construct Decision Tree ML Model

In [17]:
# Create a Decision Tree classifier object
dt = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=10, random_state=0)

# Train Decision Tree classifier
dt = dt.fit(X_train,y_train)

# Predict the response for test dataset
y_pred = dt.predict(X_test)

# Calculate how often the classifier is correct
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Balanced Accuracy:", metrics.balanced_accuracy_score(y_test, y_pred))
print("ROC AUC:", metrics.roc_auc_score(y_test, y_pred))
print("Precision Score:", metrics.precision_score(y_test, y_pred))
print("Recall Score:", metrics.recall_score(y_test, y_pred))

Accuracy: 0.9257028112449799
Balanced Accuracy: 0.8458928571428572
ROC AUC: 0.8458928571428572
Precision Score: 0.9331742243436754
Recall Score: 0.9775


In [18]:
bbb_df.iloc[:,3:].columns

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023],
      dtype='object', length=1024)

##### 3.1.1 Visualize the decision tree

In [19]:
# You don't have to code anything here.
import graphviz
import pydotplus
from sklearn import tree
 
# DOT data
dot_data = tree.export_graphviz(dt, out_file=None, 
                                feature_names=bbb_df.iloc[:,3:].columns,  
                                class_names=['0', '1'],
                                filled=True)

# Draw graph

pydot_graph = pydotplus.graph_from_dot_data(dot_data)
pydot_graph.set_size('"13,13!"')
graph = graphviz.Source(pydot_graph.to_string(), format="jpg") 
graph

ModuleNotFoundError: No module named 'graphviz'

##### 3.1.2 Generate the feature importance plot of the decision tree

In [ ]:
# You don't have to code anything here.

# get importance
dt_importance = dt.feature_importances_

# summarize and print out the feature importance
non_zero_index_list = [i for i, v in enumerate(dt_importance) if v != 0]

# remove all zero value entries
non_zero_feature_names = [bbb_df.iloc[:,3:].columns[i] for i in non_zero_index_list]
non_zero_importance = [dt_importance[i] for i in non_zero_index_list]
non_zero_feature_names_str = [str(x) for x in non_zero_feature_names]
feature_importance = pd.DataFrame(list(zip(non_zero_feature_names_str, non_zero_importance)), columns =['Feature','Importance'])

In [ ]:
fig,ax = plt.subplots(figsize=(5,10))
feature_importance.sort_values(by='Importance', ascending=True, inplace=True)
plt.xlabel('Gini Importance')
plt.ylabel('Features (with non-zero gini importance)')
plt.barh(feature_importance['Feature'], feature_importance['Importance'])
plt.title('Feature Importances')

## 4. Prediction Chemical Space Visualization

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
dim = 100
pca = PCA(n_components=dim)
components = pca.fit_transform(X_test)

In [ ]:
tsne = TSNE(n_components=2)
t_components = tsne.fit_transform(components)

In [ ]:
correct_pred = y_test != y_pred

In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.scatter(t_components[:,0], t_components[:,1], c=correct_pred, cmap='bwr', s=10)
plt.xlabel('t-SNE_1')
plt.ylabel('t-SNE_2')
plt.title('Prediction Accuracy Visualization')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.scatter(t_components[:,0], t_components[:,1], c=y_test,s=10)
plt.xlabel('t-SNE_1')
plt.ylabel('t-SNE_2')
plt.title('Permeability Visualization')
plt.colorbar()